In [ ]:
import numpy as np
from tqdm import tqdm
from constants import MAX_SEQ_LEN

with open('../hw_train.csv', 'rb') as f:
    ls = f.readlines()[1:]
# count vocabulary and max sequence length
lyrics_map = {}
bos_token = 's'.decode('utf8')
eos_token = 'e'.decode('utf8')
line_token = 'l'.decode('utf8')
none_lyrics_words = [u'編詞', u'作曲', u'作詞', u'編曲', u'監製']
vocab = [bos_token, eos_token, line_token]
lens = []
c = 0 
with tqdm(total=len(ls)) as pbar:
    for l in ls:
        l = l.decode('utf8')
        no, l = l.strip().split(',')
        no = int(no)
        if no not in lyrics_map:
            lyrics_map[no] = []
        lyrics_map[no].append(l)
        pbar.update(1)
# split into Q,K pair
with tqdm(total=len(lyrics_map)) as pbar:
    for k,v in lyrics_map.items():
        line_num = len(v)
        V = [bos_token, ]
        Q = [bos_token, ]
        for i,l in enumerate(v):
            exist_none_layrics_word = False
            for nw in none_lyrics_words:
                if nw in l:
                    exist_none_layrics_word = True
                    break
            if exist_none_layrics_word:
                continue
            
            for ws in l.strip().split(' '):
                for w in ws:
                    if i <= line_num//2:
                        V.append(w)
                    else:
                        Q.append(w)
                    vocab.append(w)
            if i <= line_num//2:
                V.append(line_token)
            else:
                Q.append(line_token)
        V.append(eos_token)
        Q.append(eos_token)
        lens.append(len(Q))
        lens.append(len(V))
        lyrics_map[k] = (Q, V)
        pbar.update(1)
            
print 'vocabulary size : %d, max len : %.0f, mean : %.0f, std : %.0f' % (len(set(vocab)), np.max(lens), np.mean(lens), np.std(lens))
print 'lyrics number : %d' % len(lyrics_map)
# print np.max(lens), np.mean(lens), np.std(lens)



 12%|█▏        | 2271/18338 [00:00<00:02, 7512.29it/s]

In [ ]:
import numpy as np
import keras
from constants import MAX_SEQ_LEN, VOCAB_DIM
from keras.preprocessing.sequence import pad_sequences

texts = vocab

tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_DIM, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=False, split=' ', char_level=True, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


In [ ]:
from constants import MAX_SEQ_LEN
from keras.preprocessing.sequence import pad_sequences

idx = np.random.permutation(len(lyrics_map))
val_num = len(lyrics_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[val_num:]}
val_u_map = {k:lyrics_map[k] for k in lyrics_map.keys()[:val_num]}
def boostrap_generator(lyrics_map, max_seq_len, tokenizer):
    while True:
        keys = lyrics_map.keys()
        for idx in np.random.permutation(len(lyrics_map)):
            no = keys[idx]
            Q, K = lyrics_map[no]
            Q = np.array([tokenizer.word_index[w] for w in Q], dtype=np.int32).reshape(1,-1)
            K = np.array([tokenizer.word_index[w] for w in K], dtype=np.int32).reshape(1,-1)
            yield Q, K
    
bat = 4
G = boostrap_generator(train_u_map, MAX_SEQ_LEN, tokenizer)
q, k = next(G)
    
print q.shape, k.shape


### Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
from constants import VOCAB_DIM, D_MODEL, MAX_SEQ_LEN
# construct neuron network
class Seq2seq_att(nn.Module):

    def __init__(self, dm, num_lay):
        super(Seq2seq_att, self).__init__()
#         for construct cache positional encoding matrix.
        self.emb = nn.Embedding(VOCAB_DIM+1, dm, padding_idx=0)
        self.encoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.decoder = nn.GRU(dm, dm, num_lay, batch_first=True)
        self.linear = nn.Linear(dm, VOCAB_DIM+1)

    def forward(self, Q, K):
        K = self.emb(K)
        Q = self.emb(Q)
        en_out, hn = self.encoder(K) 
        Q, _ = self.decoder(Q, hn)
    
        batch, q_len, _ = Q.size()
        batch, k_len, _ = K.size()
        att_in = torch.cat([en_out, Q], dim=1)
        Q_mask = self.Q_mask_matrix(batch, q_len)
        K_mask = torch.zeros([batch, q_len, k_len], dtype=torch.uint8).cuda()
        
        mask = torch.cat([K_mask, Q_mask], dim=-1)
        out = self.dot_attention(Q, att_in, att_in, mask=mask)
        
        
        
        y = self.linear(out)
        return y
    def Q_mask_matrix(self, batch, Q_len):
#         ByteTensor
        mask = torch.zeros([1, Q_len, Q_len], dtype=torch.uint8, requires_grad=False)
        for i in range(Q_len):
            mask[0,i,i+1:] = 1
        return mask.repeat(batch,1, 1).cuda()
    
    def dot_attention(self, Q, K, V, mask):
        assert Q.size()[-1] == K.size()[-1]
        assert len(Q.size()) == 3 and len(K.size()) == 3 and len(V.size()) == 3
        out = torch.matmul(Q,K.permute(0,2,1))
        if mask is not None:
            out.masked_fill_(mask, -float('inf'))
        return torch.matmul(F.softmax(out, dim=-1), V)


bat = 7
lay_num = 2
model = Seq2seq_att(D_MODEL, lay_num).cuda()
# print(o.size())
q, k = next(G)
q = torch.LongTensor(q).cuda()
k = torch.LongTensor(k).cuda()
# Q = torch.randint(VOCAB_DIM+1, [bat, MAX_SEQ_LEN]).cuda()
# K = torch.randint(VOCAB_DIM+1, [bat, MAX_SEQ_LEN]).cuda()

o = model(q, k)
print(o.size())
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



In [ ]:
print tokenizer.index_word[6456]
print 


In [ ]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%s<split>%s<split>%s<split>%s\n' % (n_iter, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label)
    log_file_stream.write(log_text.encode('utf8'))
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label):
    pred = torch.argmax(pred, dim=-1)
    mask = torch.ones_like(label, dtype=torch.uint8)
#     
    acc = pred == label
    acc = torch.sum(acc).item() / float(torch.sum(mask).item())
    
    return acc
def seq2text(output, index_word):
    assert len(output.size()) == 1
    seq = output
    s = ''
    for i in seq:
        i = int(i.item())
        if i == 0:
            continue
        w = index_word[i]
        s += w
    return s
    
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=100)
loss_q = deque(maxlen=10)

val_acc_q = deque(maxlen=100)
val_loss_q = deque(maxlen=10)

t = time.time()
best_loss = float('inf')

epochs = 100
batch_size = 1
G = boostrap_generator(train_u_map, MAX_SEQ_LEN, tokenizer)
val_G = boostrap_generator(val_u_map, MAX_SEQ_LEN, tokenizer)
# criterion = nn.CrossEntropyLoss(reduction='none')
weight = torch.ones([VOCAB_DIM+1,], dtype=torch.float)
weight[tokenizer.word_index[line_token]] = 100.

criterion = nn.CrossEntropyLoss(weight=weight.cuda())
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log-att.txt', 'w') as f:
    with open('best-att.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                q, k = next(G)
                q = torch.LongTensor(q).cuda()
                k = torch.LongTensor(k).cuda()

                q.requires_grad_(False)
                k.requires_grad_(False)


                output = model(q, k)
                y = q[:,1:]
#                 loss = torch.sum(criterion(output, y) * pad_mask[:,1:, :]) 
                loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                label = y

                pred = output[:,:-1,:,]
                acc = normal_acc(pred, label)
                acc_q.append(acc)
                train_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                train_label= seq2text(k[0,:], tokenizer.index_word)
                loss.backward()
                loss_q.append(loss.item())
                
                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    q, k = next(val_G)
                    q = torch.LongTensor(q).cuda()
                    k = torch.LongTensor(k).cuda()
                    
                    q.requires_grad_(False)
                    k.requires_grad_(False)
                    
                    output = model(q, k)
                    y = q[:,1:]
                    val_loss = criterion(output[:,:-1,:,].permute(0,2,1), y) 
                    label = y

                    pred = output[:,:-1,:,]
                    val_acc = normal_acc(pred, label)
                    val_acc_q.append(val_acc)
                    val_pred = seq2text(torch.argmax(output[0,:,:], dim=-1), tokenizer.index_word)
                    val_label= seq2text(k[0,:], tokenizer.index_word)
                    val_loss_q.append(val_loss.item())


                
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)
                loss = np.mean(acc_q)
                val_loss = np.mean(val_acc_q)
                
#                     pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f' % (acc, val_acc, loss.item(), val_loss.item()), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, val_loss, acc, val_acc, train_pred, train_label, val_pred, val_label, f,'./tmp-att.pt')
                if val_loss.item() < best_loss and it > 100:
                    torch.save(model, './best-att.pt')
                    best_loss = val_loss
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_loss))
                    best_log.flush()
                if it % 2000 == 0 and it >= 100: 
                    print 'train pred : %s\ntrain label : %s' % (train_pred, train_label)
                    print 'validation pred : %s\nvalidation label : %s' % (val_pred, val_label)

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))